# Data preparation
## Reading .csv file into Data Frame

In [56]:
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split,KFold, cross_val_score,cross_val_predict,RandomizedSearchCV,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz
from statistics import mean
from subprocess import call
from IPython.display import Image

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import time

pd.options.mode.chained_assignment = None

col_names = ['Time', 'Source', 'Destination', 'Protocol', 'Length', 'Info','src_port','dst_port']
data_frame = pd.read_csv("Samples/smallTraffic-_backup.csv",encoding = "ISO-8859-1", header=0, names=col_names)

data_frame.head(20)

,Time,Source,Destination,Protocol,Length,Info,src_port,dst_port
1,0.000000,192.168.3.131,72.14.213.138,HTTP,997,GET /complete/search?client=chrome&hl=en-US&q=...,57011.0,80.0
2,0.029841,72.14.213.138,192.168.3.131,HTTP,440,HTTP/1.1 200 OK (text/javascript),80.0,57011.0
3,0.223883,192.168.3.131,72.14.213.102,TCP,66,55950 > 80 [SYN] Seq=0 Win=8192 Len=0 MSS=14...,55950.0,80.0
4,0.229423,192.168.3.131,72.14.213.138,TCP,54,57011 > 80 [ACK] Seq=944 Ack=387 Win=16192 L...,57011.0,80.0
5,0.242649,72.14.213.102,192.168.3.131,TCP,66,"80 > 55950 [SYN, ACK] Seq=0 Ack=1 Win=5720 L...",80.0,55950.0
6,0.242696,192.168.3.131,72.14.213.102,TCP,54,55950 > 80 [ACK] Seq=1 Ack=1 Win=65780 Len=0,55950.0,80.0
7,0.243050,192.168.3.131,72.14.213.102,HTTP,998,GET /complete/search?client=chrome&hl=en-US&q=...,55950.0,80.0
8,0.262981,192.168.3.131,72.14.213.102,TCP,54,"55950 > 80 [FIN, ACK] Seq=945 Ack=1 Win=6578...",55950.0,80.0
9,0.264074,72.14.213.102,192.168.3.131,TCP,60,80 > 55950 [ACK] Seq=1 Ack=945 Win=7616 Len=0,80.0,55950.0
10,0.276671,72.14.213.102,192.168.3.131,HTTP,541,HTTP/1.1 200 OK (text/javascript),80.0,55950.0


# Data cleaning
## Set source port and destination port of ICMP to (-1) becouse of NaN occurance

In [57]:
icmp = ['ICMP']
temp_df = data_frame['Protocol'].copy(deep=True)
data_frame.src_port[temp_df.isin(icmp)] = -1 #0
data_frame.dst_port[temp_df.isin(icmp)] = -1 #0

data_frame.loc[(data_frame['src_port'] == -1) & (data_frame['dst_port'] == -1)].head(10)

,Time,Source,Destination,Protocol,Length,Info,src_port,dst_port
5960,116.676994,10.0.2.2,10.0.2.15,ICMP,202,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
5961,116.677060,10.0.2.2,10.0.2.15,ICMP,203,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6035,117.265243,10.0.2.2,10.0.2.15,ICMP,202,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6036,117.265322,10.0.2.2,10.0.2.15,ICMP,203,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6133,117.812397,10.0.2.2,10.0.2.15,ICMP,202,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6134,117.812489,10.0.2.2,10.0.2.15,ICMP,203,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6142,118.248598,10.0.2.2,10.0.2.15,ICMP,202,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6143,118.248695,10.0.2.2,10.0.2.15,ICMP,203,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6164,118.702913,10.0.2.2,10.0.2.15,ICMP,202,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0
6165,118.702988,10.0.2.2,10.0.2.15,ICMP,203,Time-to-live exceeded (Time to live exceeded i...,-1.0,-1.0


## Each ICMP packet has been succesfully changed

In [58]:
data_frame.loc[(data_frame['src_port'] != -1) & (data_frame['dst_port'] != -1) & (data_frame['Protocol'] == "ICMP")]

,Time,Source,Destination,Protocol,Length,Info,src_port,dst_port


# Data encoding
## We simplify our data from complex types into simple numbers
## Example:  "HTTP" -> 3
## This operation will speed up our algorithms a lot

In [59]:
label_enc = preprocessing.LabelEncoder()

src_encoded=label_enc.fit_transform(data_frame['Source'])
dst_encoded=label_enc.fit_transform(data_frame['Destination'])

src_port_encoded=label_enc.fit_transform(data_frame['src_port'])
dst_port_encoded=label_enc.fit_transform(data_frame['dst_port'])

len_encoded = label_enc.fit_transform(data_frame['Length'])

protocol_encoded=label_enc.fit_transform(data_frame['Protocol'])

features=list(zip(src_encoded,dst_encoded,src_port_encoded,dst_port_encoded,len_encoded))

# Searching for best parameters for Random Forest Classifier

In [60]:
X = features
y = protocol_encoded
k_fold = KFold(n_splits=10)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


rf_random.fit(X, y)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.4min finished


{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40,
 'bootstrap': True}

# Creating Random Forest Classifier based on best parameters

In [61]:
param_grid = {
 'n_estimators': [2000],
 'min_samples_split': [2],
 'min_samples_leaf': [2],
 'max_features': ['sqrt'],
 'max_depth': [20],
 'bootstrap': [True]
}

forest = RandomForestClassifier()

grid_search = GridSearchCV(estimator = forest, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

# Fitting data

In [55]:
grid_search.fit(X, y)
y_pred = grid_search.best_estimator_.predict(X)
print("accuracy:",metrics.accuracy_score(y, y_pred))

#
print("balanced_accuracy:",metrics.balanced_accuracy_score(y, y_predict))
print("precision:",metrics.precision_score(y,y_predict,average='weighted',zero_division=1))
print("f-measure:",metrics.f1_score(y,y_predict,average='weighted',zero_division=1))
print("mean_absolute_error:",metrics.mean_absolute_error(y,y_predict))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    9.3s finished


accuracy: 0.9678844400813407
balanced_accuracy: 0.4859519437977688
precision: 0.9188703394529844
f-measure: 0.9030686373511755
mean_absolute_error: 0.38980436154547365


# Fitting data to diffrent Random Forest Classifier 
## - without best parameters
## - using cross validation technique

In [17]:
randomForestClf = RandomForestClassifier()
randomForestClf.fit(X , y)

our_cross_valid_score = cross_val_score(randomForestClf, X, y, cv=k_fold, n_jobs=1,scoring='accuracy')

y_predict = cross_val_predict(randomForestClf, X, y, cv=k_fold, n_jobs=1)

print("RANDOM FOREST CLASSIFIER")
print(our_cross_valid_score)
print("\nmean : ",np.mean(our_cross_valid_score))
print("accuracy:",metrics.accuracy_score(y, y_predict))
print("balanced_accuracy:",metrics.balanced_accuracy_score(y, y_predict))
print("standard deviation:",np.std(our_cross_valid_score))
print("precision:",metrics.precision_score(y,y_predict,average='weighted',zero_division=1))
print("f-measure:",metrics.f1_score(y,y_predict,average='weighted',zero_division=1))
print("mean_absolute_error:",metrics.mean_absolute_error(y,y_predict))

RANDOM FOREST CLASSIFIER
[0.98037842 0.98457223 0.9740533  0.80154278 0.94670407 0.98036466
 0.97615708 0.95932679 0.93057504 0.84852735]

mean :  0.9382201698165318
accuracy: 0.9380828833882617
balanced_accuracy: 0.7167120475300058
standard deviation: 0.05977176338427174
precision: 0.9432571898573145
f-measure: 0.9391293128128821
mean_absolute_error: 0.18967814318771475


# Visualize single Tree from Random Forest

In [19]:
estimator = forest.estimators_[5]

# Export tree as dot file
export_graphviz(estimator, 
                out_file='Files/tree.dot', 
                rounded = True, proportion = False, 
                precision = 2, filled = True)

#Convert .dot to png
call(['dot', '-Tpng', 'Files/tree.dot', '-o', 'Files/tree.png', '-Gdpi=1200'],shell=True)

#Show tree.png
Image(filename = 'Files/tree.png')